In [0]:
# hjælp til kommandoer for filsystemet i databricks
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
dbutils.fs.rm('superusersdata',True)

Out[1]: False

In [0]:
foldername="superusersdata"
filename="animals.csv"
dbutils.fs.mkdirs(foldername)
filepath=foldername+"/"+filename
data="Id,Navn\n1,Hest\n2,Kat\n3,Hund"
dbutils.fs.put(filepath,data,True)

Wrote 27 bytes.
Out[3]: True

In [0]:
%fs ls dbfs:/superusersdata/

path,name,size,modificationTime
dbfs:/superusersdata/animals.csv,animals.csv,27,1663846923000


In [0]:
print(dbutils.fs.head(filepath))

Id,Navn
1,Hest
2,Kat
3,Hund


In [0]:
%sql
DROP TABLE animalsTable

In [0]:
# File location and type
file_location = "/superusersdata/animals.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

permanent_table_name = "animalsTable"

#df.write.format("parquet").saveAsTable(permanent_table_name)
df.write.format("delta").saveAsTable(permanent_table_name)
# default format er delta nu i den spark version vi kører her
#df.write.saveAsTable(permanent_table_name)


In [0]:
%sql
SELECT * FROM animalsTable

Id,Navn
1,Flodhest
2,Kat
3,Hund


In [0]:
%sql
INSERT INTO animalsTable
VALUES(4,'Giraf')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
DELETE FROM animalsTable
WHERE Id=4

num_affected_rows
1


In [0]:
%sql
UPDATE animalsTable
SET Navn='Flodhest'
WHERE Id=1

num_affected_rows
1


In [0]:
%sql
INSERT INTO animalsTable
VALUES(104,'Tiger')

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
DESCRIBE TABLE EXTENDED animalsTable

col_name,data_type,comment
Id,int,null
Navn,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
Table,animalsTable,
Type,MANAGED,
Location,dbfs:/user/hive/warehouse/animalstable,
Provider,delta,


In [0]:
%fs ls dbfs:/user/hive/warehouse/animalstable

path,name,size,modificationTime
dbfs:/user/hive/warehouse/animalstable/_delta_log/,_delta_log/,0,1663847482000
dbfs:/user/hive/warehouse/animalstable/part-00000-19d3801c-4917-469e-bea9-60e8806a018c-c000.snappy.parquet,part-00000-19d3801c-4917-469e-bea9-60e8806a018c-c000.snappy.parquet,830,1663847312000
dbfs:/user/hive/warehouse/animalstable/part-00000-a58f615e-4873-4b0a-a52b-2d2dfd60da3d-c000.snappy.parquet,part-00000-a58f615e-4873-4b0a-a52b-2d2dfd60da3d-c000.snappy.parquet,818,1663847274000
dbfs:/user/hive/warehouse/animalstable/part-00000-c20c3c8f-a0d2-411e-bf7f-c452d9d3c914-c000.snappy.parquet,part-00000-c20c3c8f-a0d2-411e-bf7f-c452d9d3c914-c000.snappy.parquet,816,1663847294000
dbfs:/user/hive/warehouse/animalstable/part-00000-cf50a118-a450-428d-b927-f236c5dd8129-c000.snappy.parquet,part-00000-cf50a118-a450-428d-b927-f236c5dd8129-c000.snappy.parquet,816,1663847388000


In [0]:
%sql
DROP TABLE animalsTable

In [0]:
%sql
DESCRIBE history animalsTable

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2022-09-22T11:51:20.000+0000,6814856215523555,su20220919@outlook.com,RESTORE,"Map(version -> 1, timestamp -> null)",null,List(841484036189212),0919-112737-fwgdtrpl,4,Serializable,false,"Map(numRestoredFiles -> 2, removedFilesSize -> 1646, numRemovedFiles -> 2, restoredFilesSize -> 1634, numOfFilesAfterRestore -> 2, tableSizeAfterRestore -> 1634)",null,Databricks-Runtime/11.2.x-scala2.12
4,2022-09-22T11:49:48.000+0000,6814856215523555,su20220919@outlook.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(841484036189212),0919-112737-fwgdtrpl,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 816)",null,Databricks-Runtime/11.2.x-scala2.12
3,2022-09-22T11:48:33.000+0000,6814856215523555,su20220919@outlook.com,UPDATE,Map(predicate -> (Id#1597 = 1)),null,List(841484036189212),0919-112737-fwgdtrpl,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 2, numAddedChangeFiles -> 0, executionTimeMs -> 934, scanTimeMs -> 427, numAddedFiles -> 1, numUpdatedRows -> 1, rewriteTimeMs -> 506)",null,Databricks-Runtime/11.2.x-scala2.12
2,2022-09-22T11:48:22.000+0000,6814856215523555,su20220919@outlook.com,DELETE,"Map(predicate -> [""(spark_catalog.default.animalsTable.Id = 4)""])",null,List(841484036189212),0919-112737-fwgdtrpl,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numCopiedRows -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1704, numDeletedRows -> 1, scanTimeMs -> 1424, numAddedFiles -> 0, rewriteTimeMs -> 275)",null,Databricks-Runtime/11.2.x-scala2.12
1,2022-09-22T11:48:14.000+0000,6814856215523555,su20220919@outlook.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(841484036189212),0919-112737-fwgdtrpl,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 816)",null,Databricks-Runtime/11.2.x-scala2.12
0,2022-09-22T11:47:55.000+0000,6814856215523555,su20220919@outlook.com,CREATE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(841484036189212),0919-112737-fwgdtrpl,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 818)",null,Databricks-Runtime/11.2.x-scala2.12


In [0]:
%sql
SELECT * FROM animalsTable

Id,Navn
4,Giraf
1,Hest
2,Kat
3,Hund


In [0]:
%sql
--https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-restore.html
RESTORE TABLE animalsTable TO VERSION AS OF 1

table_size_after_restore,num_of_files_after_restore,num_removed_files,num_restored_files,removed_files_size,restored_files_size
1634,2,2,2,1646,1634


In [0]:
%fs ls dbfs:/superusersdata/

path,name,size,modificationTime
dbfs:/superusersdata/animals.csv,animals.csv,27,1663846923000
dbfs:/superusersdata/animals2.csv,animals2.csv,32,1663847794000


In [0]:
%sql
DROP TABLE IF EXISTS animals;

CREATE TABLE animals
USING csv
OPTIONS (path "dbfs:/superusersdata/*.csv", header "true")

In [0]:
%sql
DESCRIBE TABLE EXTENDED animals

col_name,data_type,comment
Id,string,null
Navn,string,null
,,
# Detailed Table Information,,
Database,default,
Table,animals,
Owner,root,
Created Time,Thu Sep 22 11:55:42 UTC 2022,
Last Access,UNKNOWN,
Created By,Spark 3.3.0,


In [0]:
%sql
SELECT *
FROM   animals

Id,Navn
1,Hest
2,Kat
3,Hund


In [0]:
foldername="superusersdata"
filename="animals2.csv"
dbutils.fs.mkdirs(foldername)
filepath=foldername+"/"+filename
data="Id,Navn\n5,Flodhest\n6,Tiger\n7,And"
dbutils.fs.put(filepath,data,True)

Wrote 32 bytes.
Out[32]: True

In [0]:
%sql
REfRESH TABLE animals;

SELECT *
FROM   animals

Id,Navn
5,Flodhest
6,Tiger
7,And
1,Hest
2,Kat
3,Hund
